In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from PIL import Image
import os
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

# Stratified train-test split ensuring balanced gender representation
from sklearn.model_selection import train_test_split

In [2]:
# Read the training and test datasets
train_metadata = pd.read_csv('female_train.csv')
test_metadata_female = pd.read_csv('female_test.csv')
test_metadata_male = pd.read_csv('male_test.csv')

image_dir = 'images/all_images/'
image_files = set(os.listdir(image_dir))

# Function to check if the image exists
def image_exists(img_id):
    return img_id in image_files

# Filter the metadata to only include rows with existing images
train_metadata['image_exists'] = train_metadata['img_id'].apply(image_exists)
train_metadata_filtered = train_metadata[train_metadata['image_exists']]
train_metadata_filtered = train_metadata_filtered.drop(columns=['image_exists'])

test_metadata_female['image_exists'] = test_metadata_female['img_id'].apply(image_exists)
test_metadata_filtered_female = test_metadata_female[test_metadata_female['image_exists']]
test_metadata_filtered_female = test_metadata_filtered_female.drop(columns=['image_exists'])

test_metadata_male['image_exists'] = test_metadata_male['img_id'].apply(image_exists)
test_metadata_filtered_male = test_metadata_male[test_metadata_male['image_exists']]
test_metadata_filtered_male = test_metadata_filtered_male.drop(columns=['image_exists'])

# Map diagnostic to binary cancer labels
diagnostic_map = {'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0}
train_metadata_filtered['diagnostic'] = train_metadata_filtered['diagnostic'].map(diagnostic_map)
test_metadata_filtered_female['diagnostic'] = test_metadata_filtered_female['diagnostic'].map(diagnostic_map)
test_metadata_filtered_male['diagnostic'] = test_metadata_filtered_male['diagnostic'].map(diagnostic_map)

# Drop rows with NaN values in the 'diagnostic' column
train_metadata_filtered = train_metadata_filtered.dropna(subset=['diagnostic'])
test_metadata_filtered_female = test_metadata_filtered_female.dropna(subset=['diagnostic'])
test_metadata_filtered_male = test_metadata_filtered_male.dropna(subset=['diagnostic'])

# Create image paths
train_metadata_filtered['image_path'] = image_dir + train_metadata_filtered['img_id']
test_metadata_filtered_female['image_path'] = image_dir + test_metadata_filtered_female['img_id']
test_metadata_filtered_male['image_path'] = image_dir + test_metadata_filtered_male['img_id']

# Convert categorical features to numerical codes
categorical_cols = ['smoke', 'drink', 'skin_cancer_history', 'cancer_history', 'has_piped_water', 'has_sewage_system', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation']

for col in categorical_cols:
    train_metadata_filtered[col] = train_metadata_filtered[col].astype('category').cat.codes
    test_metadata_filtered_female[col] = test_metadata_filtered_female[col].astype('category').cat.codes
    test_metadata_filtered_male[col] = test_metadata_filtered_male[col].astype('category').cat.codes

train_df, val_df = train_test_split(train_metadata_filtered, test_size=0.2, stratify=train_metadata_filtered['gender'], random_state=42)

# Separate features and target
def prepare_data(metadata):
    X_tabular = metadata[['age', 'smoke', 'drink', 'skin_cancer_history', 'cancer_history', 'has_piped_water', 'has_sewage_system', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation']].values
    y = metadata['diagnostic'].values
    image_paths = metadata['image_path'].values
    return X_tabular, y, image_paths

X_train_tabular, y_train, train_image_paths = prepare_data(train_df)
X_val_tabular, y_val, val_image_paths = prepare_data(val_df)
X_test_tabular_female, y_test_female, test_image_paths_female = prepare_data(test_metadata_filtered_female)
X_test_tabular_male, y_test_male, test_image_paths_male = prepare_data(test_metadata_filtered_male)


In [3]:
class CustomDataset(Dataset):
    def __init__(self, tabular_data, image_paths, labels, transform=None):
        self.tabular_data = tabular_data
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        tabular_data = torch.tensor(self.tabular_data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        return tabular_data, image, label


In [4]:
# Define transforms for the image data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = CustomDataset(X_train_tabular, train_image_paths, y_train, transform=transform)
val_dataset = CustomDataset(X_val_tabular, val_image_paths, y_val, transform=transform)
test_dataset_female = CustomDataset(X_test_tabular_female, test_image_paths_female, y_test_female, transform=transform)
test_dataset_male = CustomDataset(X_test_tabular_male, test_image_paths_male, y_test_male, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader_female = DataLoader(test_dataset_female, batch_size=32, shuffle=False)
test_loader_male = DataLoader(test_dataset_male, batch_size=32, shuffle=False)

In [5]:
class MultimodalNetwork(nn.Module):
    def __init__(self):
        super(MultimodalNetwork, self).__init__()
        
        # ResNet18 model for image data
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity()  # Remove the final layer
        
        # Multi Layer Perceptron for tabular data
        self.fc_tabular = nn.Sequential(
            nn.Linear(14, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU()
        )
        
        # Combined layers
        self.fc_combined = nn.Sequential(
            nn.Linear(512 + 32, 128),  # Adjusted for ResNet18 output size
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def forward(self, tabular_data, images):
        img_features = self.resnet(images)
        tabular_features = self.fc_tabular(tabular_data)
        combined_features = torch.cat((img_features, tabular_features), dim=1)
        output = self.fc_combined(combined_features)
        return output


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultimodalNetwork().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100  # Set a high number of epochs
early_stop_threshold = 0.03  # Minimum loss decrease threshold
early_stop_patience = 5  # Number of epochs to wait for improvement

best_loss = float('inf')
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for tabular_data, images, labels in train_loader:
        tabular_data, images, labels = tabular_data.to(device), images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(tabular_data, images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * tabular_data.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # Early stopping logic
    if epoch_loss < best_loss - early_stop_threshold:
        best_loss = epoch_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience or epoch_loss < 0.1:
        print(f'Early stopping at epoch {epoch+1}')
        break


c:\Users\umutg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\umutg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100, Loss: 0.6467
Epoch 2/100, Loss: 0.5050
Epoch 3/100, Loss: 0.4067
Epoch 4/100, Loss: 0.3527
Epoch 5/100, Loss: 0.2474
Epoch 6/100, Loss: 0.1480
Epoch 7/100, Loss: 0.3861
Epoch 8/100, Loss: 0.2881
Epoch 9/100, Loss: 0.2191
Epoch 10/100, Loss: 0.1576
Epoch 11/100, Loss: 0.0999
Early stopping at epoch 11


In [7]:
def evaluate_model(test_loader, model, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for tabular_data, images, labels in test_loader:
            tabular_data, images, labels = tabular_data.to(device), images.to(device), labels.to(device)
            
            outputs = model(tabular_data, images).squeeze()
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * tabular_data.size(0)
            predicted = (outputs > 0.5).float()
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss = test_loss / len(test_loader.dataset)
    accuracy = correct / total
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    auc_roc = roc_auc_score(all_labels, all_predictions)
    report = classification_report(all_labels, all_predictions)

    return test_loss, accuracy, conf_matrix, auc_roc, report

# Evaluate on female test dataset
test_loss_female, accuracy_female, conf_matrix_female, auc_roc_female, report_female = evaluate_model(test_loader_female, model, criterion)
print(f'Female Test Loss: {test_loss_female:.4f}, Accuracy: {accuracy_female:.4f}, AUC-ROC: {auc_roc_female:.4f}')
print('Female Confusion Matrix:')
print(conf_matrix_female)
print('Female Classification Report:')
print(report_female)

# Evaluate on male test dataset
test_loss_male, accuracy_male, conf_matrix_male, auc_roc_male, report_male = evaluate_model(test_loader_male, model, criterion)
print(f'Male Test Loss: {test_loss_male:.4f}, Accuracy: {accuracy_male:.4f}, AUC-ROC: {auc_roc_male:.4f}')
print('Male Confusion Matrix:')
print(conf_matrix_male)
print('Male Classification Report:')
print(report_male)

Female Test Loss: 2.0654, Accuracy: 0.7816, AUC-ROC: 0.7979
Female Confusion Matrix:
[[28 19]
 [ 0 40]]
Female Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.60      0.75        47
         1.0       0.68      1.00      0.81        40

    accuracy                           0.78        87
   macro avg       0.84      0.80      0.78        87
weighted avg       0.85      0.78      0.77        87

Male Test Loss: 1.7525, Accuracy: 0.6207, AUC-ROC: 0.6184
Male Confusion Matrix:
[[18 25]
 [ 8 36]]
Male Classification Report:
              precision    recall  f1-score   support

         0.0       0.69      0.42      0.52        43
         1.0       0.59      0.82      0.69        44

    accuracy                           0.62        87
   macro avg       0.64      0.62      0.60        87
weighted avg       0.64      0.62      0.60        87

